#### Unitycatalog-langchain E2E example
Import and run this notebook in Databricks

In [0]:
%pip install unitycatalog-langchain[databricks] mlflow langchain_openai
%restart_python

#### Create a DatabricksFunctionClient

Create a client instance in order to create and execute functions using Databricks serverless

In [0]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# set the default uc function client, it will be used for all toolkits
set_uc_function_client(client)

#### Create a UC function for executing python code

In [0]:
# replace with your own catalog and schema
CATALOG = "ml"
SCHEMA = "serena_test"


def execute_python_code(code: str) -> str:
    """
    Executes the given python code and returns its stdout.

    Args:
      code: Python code to execute. Remember to print the final result to stdout.
    """
    import sys
    from io import StringIO

    stdout = StringIO()
    sys.stdout = stdout
    exec(code)
    return stdout.getvalue()


function_info = client.create_python_function(
    func=execute_python_code, catalog=CATALOG, schema=SCHEMA, replace=True
)
python_execution_function_name = function_info.full_name

In [0]:
client.execute_function(python_execution_function_name, {"code": "print(1+1)"})

FunctionExecutionResult(error=None, format='SCALAR', value='2\n', truncated=None)

#### Create a UC function for translation

Use Databricks built-in [ai_translate](https://docs.databricks.com/en/sql/language-manual/functions/ai_translate.html) function

In [0]:
translate_function_name = f"{CATALOG}.{SCHEMA}.translate"
sql_body = f"""CREATE FUNCTION {translate_function_name}(content STRING COMMENT 'the text to be translated', language STRING COMMENT 'the target language code to translate the content to')
RETURNS STRING
RETURN SELECT ai_translate(content, language)
"""
client.create_function(sql_function_body=sql_body)

FunctionInfo(catalog_name='ml', comment=None, created_at=1729134762348, created_by='serena.ruan@databricks.com', data_type=<ColumnTypeName.STRING: 'STRING'>, external_language=None, external_name=None, full_data_type='STRING', full_name='ml.serena_test.translate', function_id='c00867e8-c4a8-499b-86ff-75522d287e01', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='content', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment='the text to be translated', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"content","type":"string","nullable":true,"metadata":{"comment":"the text to be translated"}}', type_precision=0, type_scale=0), FunctionParameterInfo(name='language', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=1, comment='the target language code to translate the content to', parameter_default=None, parame

In [0]:
client.execute_function(
    translate_function_name, {"content": "What is Databricks?", "language": "es"}
)

FunctionExecutionResult(error=None, format='SCALAR', value='¿Qué es Databricks?', truncated=None)

#### Create tools

In [0]:
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(
    function_names=[python_execution_function_name, translate_function_name]
)
tools = toolkit.tools
tools

/local_disk0/.ephemeral_nfs/envs/pythonEnv-4d758604-2ab1-4b55-b71d-b091672de6c3/lib/python3.10/site-packages/ucai_langchain/toolkit.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_core.pydantic_v1 import BaseModel, Field, root_validator


[UnityCatalogTool(name='ml__serena_test__execute_python_code', description='Executes the given python code and returns its stdout.', args_schema=<class 'ucai.core.utils.function_processing_utils.ml__serena_test__execute_python_code__params'>, func=<function UCFunctionToolkit.uc_function_to_langchain_tool.<locals>.func at 0x7fb311093b50>, client_config={'warehouse_id': None, 'profile': None}),
 UnityCatalogTool(name='ml__serena_test__translate', args_schema=<class 'ucai.core.utils.function_processing_utils.ml__serena_test__translate__params'>, func=<function UCFunctionToolkit.uc_function_to_langchain_tool.<locals>.func at 0x7fb30f82d7e0>, client_config={'warehouse_id': None, 'profile': None})]

#### Use the tools in Langchain Agent

In [0]:
import os

# Replace with your own API key
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [0]:
agent_executor.invoke({"input": "What is the value of 2**10?"})



> Entering new AgentExecutor chain...

Invoking: `ml__serena_test__execute_python_code` with `{'code': 'print(2**10)'}`


{"format": "SCALAR", "value": "1024\n"}The value of \( 2^{10} \) is 1024.

> Finished chain.


{'input': 'What is the value of 2**10?',
 'output': 'The value of \\( 2^{10} \\) is 1024.'}

In [0]:
agent_executor.invoke({"input": "What is English for '我是一个机器人'?"})



> Entering new AgentExecutor chain...

Invoking: `ml__serena_test__translate` with `{'content': '我是一个机器人', 'language': 'en'}`


{"format": "SCALAR", "value": "I am a robot"}The English translation for '我是一个机器人' is 'I am a robot'.

> Finished chain.


{'input': "What is English for '我是一个机器人'?",
 'output': "The English translation for '我是一个机器人' is 'I am a robot'."}